<a href="https://colab.research.google.com/github/damilola2010/2022-Tweet-Sentiment-Analysis/blob/main/2022_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The aim of this project was to use natural language processing techniques to conduct sentiment analysis on Twitter data. In January 2022, I wanted to gain insight into the feelings of Twitter users about the year. To complete the project, a number of Python libraries were utilized, including Pandas, Tweepy, NLTK, TextBlob, MatPlotlib, WordCloud, Emot, and Plotly. These libraries were used for tasks such as data cleaning, mining tweets, sentiment analysis, data exploration, identifying emojis, and data visualization.

### Install Necessary Lib and Packages

In [1]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 11.7 MB/s eta 0:00:00


In [2]:
  >>> import nltk
  >>> nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import tweepy
import re
from textblob import TextBlob
import matplotlib.pyplot as plt

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
import csv

In [6]:

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [7]:
import glob # to retrieve files/pathnames matching a specified pattern. 
import random # generating random numbers
import pandas as pd # for data manipulation and analysis
import requests # to send HTTP requests
from PIL import Image # for opening, manipulating, and saving many different image file f
import numpy as np # for working with arrays and carrying out mathematical operations. Pandas is built on Numpy
import string # Inbuilt string library

# Mining Tweets

In [8]:
# Putting Twitter Api keys and Tokens
consumer_key = "Wqdwq2M7tkOVvY5lUTbV4feQ8"
consumer_secret = "KyeJFbGYNzWxR0s5nUe97aLBSFEF17cKQ9EgSQMBu1Yz2DZp8m"
access_token = "1321314967812034566-9vFJrCLeNr8sgGlOcsyk7cw71luyvu"
access_token_secret = "ERbnjWJ6rumv24meh9JxrqS2j6b3XBzP6zHpaDn5V91r9"


In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) # Pass in Consumer key and secret for authentication by API
auth.set_access_token(access_token, access_token_secret) # Pass in Access key and secret for authentication by API
api = tweepy.API(auth) 

In [13]:
tweet_list = [tweet for tweet in tweepy.Cursor(api.search,
                                q="2022 was a",
                                lang="en",
                                since_id="2022-12-12",
                                tweet_mode='extended').items(2000)]

## Processing Tweets

In [14]:

# Initialize the stemmer and stop words list
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

# Pre-process the tweets
processed_tweets = []
for tweet in tweet_list:
    # Remove special characters, URLs, and hashtags
    text = re.sub(r'[^a-zA-Z0-9]', ' ', tweet.full_text)
    text = re.sub(r'https?://\S+', ' ', text)
    text = re.sub(r'#', ' ', text)
    
    # Lowercase and split the text into words
    words = text.lower().split()
    
    # Remove stop words and stem the remaining words
    stemmed_words = [stemmer.stem(word) for word in words if word not in stop_words]
    
    # Join the stemmed words back into a single string
    processed_tweet = ' '.join(stemmed_words)
    


## Output Processed tweets to CSV

In [16]:


# Open the CSV file in write mode
with open('processed_tweets.csv', 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer object
    writer = csv.writer(csv_file, delimiter=',')
    
  # Write the column names
    writer.writerow(['tweet_id', 'created_at', 'text', 'location', 'likes'])
    
    # Iterate through the processed tweets and write each row to the CSV file
    for tweet in tweet_list:
        tweet_id = tweet.id
        created_at = tweet.created_at
        text = tweet.full_text
        location = tweet.user.location
        likes = tweet.favorite_count
        writer.writerow([tweet_id, created_at, text, location, likes])

In [17]:
# Load the CSV file into a DataFrame
df = pd.read_csv("processed_tweets.csv")

# Print the first few rows of the DataFrame
df.head()

,tweet_id,created_at,text,location,likes
0,1611282681345757185,2023-01-06 08:45:02,@yakyucosmo I know there are a lot of opinions...,誰よりも侍っぽいスティーブン・クワン激推し🏟,0
1,1611282672495792128,2023-01-06 08:44:59,RT @On3sports: 🚨BREAKING🚨 Jackson State WR tra...,On The Field 🏈,0
2,1611282634243559424,2023-01-06 08:44:50,RT @thenewsmovement: Morning everyone! This is...,"West Midlands, England",0
3,1611282595433660416,2023-01-06 08:44:41,RT @mingaya_TEEZ: just wanted to remind everyo...,They/He/She/Any,0
4,1611282592539582466,2023-01-06 08:44:40,RT @thekhrc: This is a frightening crime but i...,Nairobi,0


## Exploratory Data Analysis

In [19]:
df.shape #Get number of rows and columns

(2000, 5)

In [20]:
df.duplicated(subset='tweet_id').sum() # Check for duplicate values

0

In [21]:
df.isna().any() # Check for "NaN" values

tweet_id      False
created_at    False
text          False
location       True
likes         False
dtype: bool

In [22]:
df['location']=df['location'].fillna('No location') # Replace "NaN" values with "No Location"

In [23]:
df.isna().any() # Check for "NaN" values

tweet_id      False
created_at    False
text          False
location      False
likes         False
dtype: bool

In [24]:
import plotly.express as px

# Group the DataFrame by location and count the number of tweets for each location
location_counts = df.groupby("location").size().reset_index(name="count")

# Creating a choropleth map showing the number of tweets per location
fig = px.choropleth(location_counts, 
                    locations="location", 
                    color="count",
                    title="Number of Tweets per Location",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()


## Sentiment Analysis